In [5]:
from ultralytics import YOLO
import pandas as pd
import numpy as np
from pathlib import Path
import cv2, os, sys

# --------------------
# CONFIG
# --------------------
POSE_CONF_THRESHOLD = 0.8  # reject detections below this avg confidence
model = YOLO("/home/debasish/Documents/YOLOv8/weights/pose-weights/yolov8x-832.pt")

# Input folder or single file
source = Path("/home/debasish/Documents/YOLOv8/videos/count.png")

# --------------------
# Setup Output Directory
# --------------------
base_out_dir = Path("outputs/pose")
base_out_dir.mkdir(parents=True, exist_ok=True)

# auto-increment run folder: run1, run2, run3 ...
existing_runs = [d for d in base_out_dir.iterdir() if d.is_dir() and d.name.startswith("run")]
run_num = len(existing_runs) + 1
run_dir = base_out_dir / f"run{run_num}"
run_dir.mkdir(parents=True, exist_ok=True)

print(f"Outputs will be saved in: {run_dir}")

# --------------------
# Collect Files
# --------------------
if source.is_dir():
    files = list(source.glob("*.*"))  # all files in folder
else:
    files = [source]

video_exts = [".mp4", ".avi", ".mov", ".mkv"]
image_exts = [".jpg", ".jpeg", ".png", ".bmp"]

# --------------------
# Process Each File
# --------------------
for file_path in files:
    ext = file_path.suffix.lower()
    filename = file_path.stem

    out_video_path = run_dir / f"{filename}.avi"
    out_img_path = run_dir / f"{filename}.jpg"
    out_csv_path = run_dir / f"{filename}.csv"

    print(f"\n Processing: {file_path.name}")

    # Run tracking
    results = model.track(
        str(file_path),
        persist=True,
        show=False,
        show_labels=False,
        show_conf=False,
        tracker="botsort.yaml",
    )

    # Tracking vars
    frame = 0
    frames = []; obj_ids = []; obj_clss = []
    x = []; y = []; w = []; h = []; key_pts = []

    # Video setup
    is_video = ext in video_exts
    if is_video:
        cap = cv2.VideoCapture(str(file_path))
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        writer = cv2.VideoWriter(str(out_video_path), fourcc, fps, (width, height))

        if not writer.isOpened():
            raise RuntimeError(f"Failed to open VideoWriter: {out_video_path}")
        print(f"VideoWriter opened: {out_video_path}")
    else:
        writer = None

    # Frame loop
    for result in results:
        if is_video:
            ret, frame_img = cap.read()
            if not ret:
                break
        else:
            frame_img = cv2.imread(str(file_path))
            if frame_img is None:
                raise RuntimeError(f"Failed to read image: {file_path}")

        if result.boxes is None or result.keypoints is None:
            if writer: writer.write(frame_img)
            continue

        boxes = result.boxes.cpu().numpy()
        keypoints = result.keypoints.cpu().numpy()

        if boxes is None or len(boxes) == 0 or keypoints is None or keypoints.xy is None or len(keypoints.xy) == 0:
            if writer: writer.write(frame_img)
            continue
        if boxes.cls is None or boxes.id is None or boxes.xywh is None:
            if writer: writer.write(frame_img)
            continue

        obj_cls = boxes.cls
        obj_id = boxes.id
        xywh = boxes.xywh
        kpts_xy = keypoints.xy
        kpts_conf = keypoints.conf

        frame += 1
        for i, j, k, l, c in zip(obj_cls, obj_id, xywh, kpts_xy, kpts_conf):
            avg_conf = np.mean(c)

            if avg_conf < POSE_CONF_THRESHOLD:
                continue  # skip low confidence

            obj_id_int = int(j)
            frames.append(frame)
            obj_clss.append(int(i))
            obj_ids.append(obj_id_int)

            x.append(k[0]); y.append(k[1]); w.append(k[2]); h.append(k[3])
            key_pts.append(l.reshape(-1))

            # --- draw keypoints ---
            num_kpts = l.shape[0]
            for idx, (px, py) in enumerate(l.astype(int)):
                r = 0
                g = int(255 * (idx / (num_kpts - 1))) if num_kpts > 1 else 255
                b = int(255 * (1 - idx / (num_kpts - 1))) if num_kpts > 1 else 0
                color = (b, g, r)
                cv2.circle(frame_img, (px, py), 3, color, -1)

        if writer:
            writer.write(frame_img)
        else:
            cv2.imwrite(str(out_img_path), frame_img)

    # Cleanup
    if is_video:
        cap.release()
        writer.release()
        print(f"Video saved at: {out_video_path}")
    else:
        print(f"Image saved at: {out_img_path}")

    # Save CSV
    data = {"frame": frames, "obj_cls": obj_clss, "obj_id": obj_ids}
    kpts_out1 = pd.DataFrame(data)
    bbox_out = {"x": x, "y": y, "w": w, "h": h}
    out1 = pd.DataFrame(bbox_out)
    kpts_out2 = pd.DataFrame(key_pts)

    out = pd.concat([kpts_out1, out1, kpts_out2], axis=1)
    out.to_csv(out_csv_path, index=False)
    print(f"CSV saved at: {out_csv_path}")

Outputs will be saved in: outputs/pose/run4

 Processing: count.png

image 1/1 /home/debasish/Documents/YOLOv8/videos/count.png: 480x832 11 worms, 19.6ms
Speed: 1.5ms preprocess, 19.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 832)
Image saved at: outputs/pose/run4/count.jpg
CSV saved at: outputs/pose/run4/count.csv
